# Import libraries

In [1]:
import data_augmentation
from scipy.io import wavfile as wav
import IPython
import librosa
import os

/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


# Test data augmentation functions
## Random noise
### Original audio:

In [2]:
fn= "output/0_khaled_0.wav"
_, original_signal = wav.read(fn)
rate=8000
IPython.display.Audio(original_signal,rate=rate)

### Upper bound

In [3]:
noise_audio = data_augmentation.add_random_noise(original_signal, mu=0, stdev=0.1)

In [4]:
IPython.display.Audio(noise_audio,rate=rate)

A standard deviation equal to 0.1 may be too agressive: the audio is intelligible but there is a lot of noise background. Let's see what happens with 0.05:

In [5]:
noise_audio = data_augmentation.add_random_noise(original_signal, mu=0, stdev=0.05)
IPython.display.Audio(noise_audio,rate=rate)

It sounds better! For the moment let's keep 0.05 as upper bound standard deviation.  Now let's find the lower bound
### Lower bound

In [6]:
noise_audio = data_augmentation.add_random_noise(original_signal, mu=0, stdev=0.001)
IPython.display.Audio(noise_audio,rate=rate)

The noise can be heard distinctly only in the last part. Let's increase it to 0.005

In [7]:
noise_audio = data_augmentation.add_random_noise(original_signal, mu=0, stdev=0.005)
IPython.display.Audio(noise_audio,rate=rate)

Now the noise can be heard also during the speaking part.

**OPEN POINT: we keep 0.001 or 0.005 as lower bound?**

## PITCH SHIFT

## Original audio

In [8]:
IPython.display.Audio(original_signal,rate=rate)

### Lower bound

In [9]:
pitch_shift_audio = data_augmentation.change_pitch(original_signal, sampling_rate=rate, pitch_step = -5)
IPython.display.Audio(pitch_shift_audio,rate=rate)

Nica caveman voice :) Let's try with -10:

In [10]:
pitch_shift_audio = data_augmentation.change_pitch(original_signal, sampling_rate=rate, pitch_step = -10)
IPython.display.Audio(pitch_shift_audio,rate=rate)

The voice is incomprehensible. let's try with a pitch step of -7

In [11]:
pitch_shift_audio = data_augmentation.change_pitch(original_signal, sampling_rate=rate, pitch_step = -7)
IPython.display.Audio(pitch_shift_audio,rate=rate)

And this is -6:

In [12]:
pitch_shift_audio = data_augmentation.change_pitch(original_signal, sampling_rate=rate, pitch_step = -6)
IPython.display.Audio(pitch_shift_audio,rate=rate)

**This is borderline: what do we do?** -7 may be a bit aggressive
### Upper bound

In [13]:
pitch_shift_audio = data_augmentation.change_pitch(original_signal, sampling_rate=rate, pitch_step = 5)
IPython.display.Audio(pitch_shift_audio,rate=rate)

In [14]:
pitch_shift_audio = data_augmentation.change_pitch(original_signal, sampling_rate=rate, pitch_step = 4)
IPython.display.Audio(pitch_shift_audio,rate=rate)

In [15]:
pitch_shift_audio = data_augmentation.change_pitch(original_signal, sampling_rate=rate, pitch_step = 6)
IPython.display.Audio(pitch_shift_audio,rate=rate)

**5 seems a good compromise, what do you think?**

# Enrich dataset

## Params

In [16]:
MIN_STDEV = 0.001
MAX_STDEV = 0.05
MIN_PITCH_STEP = -6
MAX_PITCH_STEP = 5
SAMPLING_RATE = 8000

In [17]:
MIN_PITCH_STEP

-6

## Enrich dataset Wrapper function

In [18]:
MIN_PITCH_STEP

-6

In [19]:
MAX_PITCH_STEP

5

In [20]:
def enrich_dataset(audio_dir, mode, n_noise, n_pitch, rate=8000):
    enriched_audio_tracks = {}
    for audio_fn in os.listdir(audio_dir):
        # Skip temporary files
        if audio_fn.endswith(".wav"):
            # _, original_signal = wav.read(os.path.join(audio_dir,audio_fn))
            original_signal, _ = librosa.core.load(os.path.join(audio_dir,audio_fn), rate)
            # Create an empty dict for storing the various tracks associated with the current file
            enriched_audio_tracks[audio_fn] = {}
            # Add the current audio
            enriched_audio_tracks[audio_fn]['original'] = [original_signal]
            # Apply various random noises to the original track
            noise_tracks = data_augmentation.augment_audio_with_random_noise(original_signal, MIN_STDEV, MAX_STDEV, n_noise)
            # Add these tracks to the result dictionary
            enriched_audio_tracks[audio_fn]['noise'] = noise_tracks
            # Apply pitch shift only to the original audio
            current_pitch_tracks = data_augmentation.augment_audio_with_pitch_shift(audio_signal=original_signal,
                                                             sampling_rate=SAMPLING_RATE,
                                                             min_pitch_shift=MIN_PITCH_STEP,
                                                             max_pitch_shift=MAX_PITCH_STEP,
                                                             n=n_pitch)
            # Store them
            enriched_audio_tracks[audio_fn]['pitch'] = current_pitch_tracks
            if mode == "all_combinations":
                # Create a list for storing the tracks obtained through pitch shift
                pitch_noise_tracks = []
                # Iterate on the list with noise tracks
                for track in noise_tracks:
                    
                    current_pitch_tracks = data_augmentation.augment_audio_with_pitch_shift(track,
                                                                 SAMPLING_RATE,
                                                                 MIN_PITCH_STEP,
                                                                 MAX_PITCH_STEP,
                                                                 n_pitch)
                    pitch_noise_tracks = pitch_noise_tracks + current_pitch_tracks
                # Add the tracks enriched with pitch shift to the current list
                enriched_audio_tracks[audio_fn]['pitch_noise'] = pitch_noise_tracks

    return enriched_audio_tracks


## Test
### Original tracks

In [21]:
audio_dir = "./output/"
recs = os.listdir(audio_dir)
recs = [rec for rec in recs if rec.endswith(".wav")]
n_recs = len(recs)
n_recs

300

### Expected results:

mode = *all_combinations*

n_noise = 5

n_pitch = 5

Number of recordings = 300 + 5 * 300 (NOISE on original) + 5 * 300 (SHIFT on original) + 5 * (5 * 300) (SHIFT applied to NOISE) = 3300 + 5 * 1500 = 10800

Let's check the results:

In [22]:
%%time
all_comb_augmentation = enrich_dataset(audio_dir,
                                       mode="all_combinations",
                                       n_noise=5,
                                       n_pitch=5)

CPU times: user 1min 12s, sys: 3.11 s, total: 1min 15s
Wall time: 1min 20s


Let's compute the number of tracks

In [23]:
n_tracks = 0
for fn in all_comb_augmentation.keys():
    n_tracks += len(all_comb_augmentation[fn]['original'])
    n_tracks += len(all_comb_augmentation[fn]['noise'])
    n_tracks += len(all_comb_augmentation[fn]['pitch'])
    n_tracks += len(all_comb_augmentation[fn]['pitch_noise'])

In [24]:
n_tracks

10800

The results seem good! Let's try with the other augmentation mode.

mode = *normal*

n_noise = 5

n_pitch = 5

Number of recordings = 300 + 5 * 300 (NOISE on original) + 5 * 300 (SHIFT on original) = 3300

In [25]:
%%time
normal_augmentation = enrich_dataset(audio_dir,
                                     mode="normal",
                                     n_noise=5,
                                     n_pitch=5)

CPU times: user 13.8 s, sys: 669 ms, total: 14.4 s
Wall time: 17.5 s


In [26]:
n_tracks = 0
for fn in normal_augmentation.keys():
    n_tracks += len(normal_augmentation[fn]['original'])
    n_tracks += len(normal_augmentation[fn]['noise'])
    n_tracks += len(normal_augmentation[fn]['pitch'])

In [27]:
n_tracks

3300

## Store augmented dataset

In [28]:
def store_list_recordings(track_list, augmentation_strategy, augmentation_dir, base_fn):
    extension = ".wav"
    augmentation_fn_structure = "{directory}/{base_fn}_{augmentation_strategy}_{n}{extension}"
    for i,track in enumerate(track_list):
        librosa.output.write_wav(augmentation_fn_structure.format(directory=augmentation_dir,
                                                                  base_fn=base_fn,
                                                                  augmentation_strategy=augmentation_strategy,
                                                                  n=i,
                                                                  extension=extension),
                                 track,
                                 SAMPLING_RATE)
def store_recordings(augmentation_dir, recordings):
    extension = ".wav"
    for original_fn in recordings.keys():
        current_fn_recordings = recordings[original_fn]
        base_fn = original_fn.replace(extension,"")
        #Store original recording
        librosa.output.write_wav("{directory}/{fn}".format(directory=augmentation_dir,
                                                           fn=original_fn),
                                 current_fn_recordings['original'][0],
                                 SAMPLING_RATE)
        # Store noise recordings
        store_list_recordings(current_fn_recordings['noise'], "noise", augmentation_dir, base_fn)
        #Store pitch recordings
        store_list_recordings(current_fn_recordings['pitch'], "pitch", augmentation_dir, base_fn)
        # Check if combination of pitch and noise were created
        if "pitch_noise" in current_fn_recordings.keys():
            store_list_recordings(current_fn_recordings['pitch_noise'], "pitch_noise", augmentation_dir, base_fn)
        

### Store "normal" augmented recordings

In [29]:
! mkdir ./augmentation_recs

In [30]:
store_recordings("./augmentation_recs", normal_augmentation)

In [31]:
! ls ./augmentation_recs | wc -l

    3300


### Store "all combinations" augmented recordings

In [32]:
! mkdir ./all_combination

In [33]:
store_recordings("./all_combination", all_comb_augmentation)

In [34]:
! ls ./all_combination | wc -l

   10800


### Zip recordings
(In case of need)

In [39]:
!zip -r normal_augmentation.zip ./augmentation_recs
!mv normal_augmentation.zip ./augmentation_recs

  adding: augmentation_recs/ (stored 0%)
  adding: augmentation_recs/0_alinda_4_pitch_2.wav (deflated 7%)
  adding: augmentation_recs/3_alinda_2_noise_3.wav (deflated 4%)
  adding: augmentation_recs/5_gian_4.wav (deflated 39%)
  adding: augmentation_recs/9_alinda_3_pitch_0.wav (deflated 7%)
  adding: augmentation_recs/3_gian_8_noise_1.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_4_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/1_khaled_8_pitch_3.wav (deflated 6%)
  adding: augmentation_recs/7_khaled_6_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/4_khaled_0_noise_1.wav (deflated 4%)
  adding: augmentation_recs/3_khaled_1_noise_3.wav (deflated 4%)
  adding: augmentation_recs/0_khaled_7_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/9_khaled_0_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/1_gian_6_noise_4.wav (deflated 4%)
  adding: augmentation_recs/6_khaled_9_pitch_1.wav (deflated 6%)
  adding: augmentation_recs/7_gian_0_noise_0.wav (deflated 4%)

  adding: augmentation_recs/7_gian_8_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/5_khaled_7_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/0_gian_0.wav (deflated 39%)
  adding: augmentation_recs/8_khaled_7_noise_3.wav (deflated 4%)
  adding: augmentation_recs/2_gian_2_pitch_4.wav (deflated 7%)
  adding: augmentation_recs/2_khaled_6_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/1_khaled_0_noise_1.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_8_pitch_3.wav (deflated 6%)
  adding: augmentation_recs/1_khaled_4_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/2_gian_6_noise_1.wav (deflated 4%)
  adding: augmentation_recs/4_gian_4_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/3_gian_0_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/5_gian_2_noise_2.wav (deflated 4%)
  adding: augmentation_recs/5_alinda_4_pitch_2.wav (deflated 7%)
  adding: augmentation_recs/6_alinda_2_noise_3.wav (deflated 4%)
  adding: augmentation_recs/3_gian_1_noise_3.w

  adding: augmentation_recs/7_khaled_7_noise_3.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_1_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/0_alinda_5_noise_1.wav (deflated 4%)
  adding: augmentation_recs/3_alinda_3_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/9_alinda_2_noise_3.wav (deflated 4%)
  adding: augmentation_recs/0_alinda_1_pitch_4.wav (deflated 7%)
  adding: augmentation_recs/3_gian_9_pitch_2.wav (deflated 7%)
  adding: augmentation_recs/9_khaled_3.wav (deflated 44%)
  adding: augmentation_recs/3_alinda_7_noise_4.wav (deflated 4%)
  adding: augmentation_recs/9_khaled_2.wav (deflated 43%)
  adding: augmentation_recs/3_gian_9_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/9_alinda_2_noise_2.wav (deflated 4%)
  adding: augmentation_recs/0_alinda_5_noise_0.wav (deflated 4%)
  adding: augmentation_recs/3_alinda_3_pitch_1.wav (deflated 6%)
  adding: augmentation_recs/7_khaled_7_noise_2.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_1_pitc

  adding: augmentation_recs/1_khaled_5_noise_3.wav (deflated 4%)
  adding: augmentation_recs/8_khaled_2_noise_1.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_9_noise_4.wav (deflated 4%)
  adding: augmentation_recs/2_gian_3_noise_3.wav (deflated 4%)
  adding: augmentation_recs/8_khaled_6_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/4_gian_1_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/2_gian_2_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/8_khaled_7_noise_4.wav (deflated 4%)
  adding: augmentation_recs/4_gian_0_noise_2.wav (deflated 4%)
  adding: augmentation_recs/1_khaled_4_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/2_khaled_2_noise_2.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_8_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/8_khaled_3_pitch_1.wav (deflated 7%)
  adding: augmentation_recs/3_gian_4_noise_1.wav (deflated 4%)
  adding: augmentation_recs/6_alinda_6_pitch_1.wav (deflated 6%)
  adding: augmentation_recs/5_alind

  adding: augmentation_recs/1_gian_6_noise_3.wav (deflated 4%)
  adding: augmentation_recs/9_khaled_4_noise_2.wav (deflated 4%)
  adding: augmentation_recs/0_khaled_3_noise_0.wav (deflated 4%)
  adding: augmentation_recs/3_khaled_5_pitch_1.wav (deflated 6%)
  adding: augmentation_recs/8_gian_3_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/6_gian_2_noise_0.wav (deflated 4%)
  adding: augmentation_recs/7_alinda_1_noise_2.wav (deflated 4%)
  adding: augmentation_recs/4_alinda_7_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/9_gian_1_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/0_gian_0_pitch_1.wav (deflated 7%)
  adding: augmentation_recs/2_alinda_9_pitch_0.wav (deflated 7%)
  adding: augmentation_recs/0_gian_4_noise_4.wav (deflated 4%)
  adding: augmentation_recs/8_alinda_8_noise_3.wav (deflated 4%)
  adding: augmentation_recs/5_alinda_8_pitch_2.wav (deflated 7%)
  adding: augmentation_recs/5_gian_3.wav (deflated 38%)
  adding: augmentation_recs/3_alinda_2_noise_4

...

  adding: augmentation_recs/2_alinda_2_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/8_alinda_7_pitch_2.wav (deflated 7%)
  adding: augmentation_recs/2_alinda_6_noise_1.wav (deflated 4%)
  adding: augmentation_recs/1_alinda_0_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/6_gian_9_noise_4.wav (deflated 4%)
  adding: augmentation_recs/1_gian_9_pitch_2.wav (deflated 7%)
  adding: augmentation_recs/8_gian_8_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/0_khaled_8_noise_4.wav (deflated 4%)
  adding: augmentation_recs/5_khaled_4_noise_3.wav (deflated 4%)
  adding: augmentation_recs/6_khaled_2_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/4_gian_7_noise_1.wav (deflated 4%)
  adding: augmentation_recs/2_khaled_1_pitch_4.wav (deflated 7%)
  adding: augmentation_recs/2_gian_5_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/4_gian_3_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/8_khaled_4_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/1_khaled_

  adding: augmentation_recs/9_alinda_0_noise_1.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_3_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/7_khaled_5_noise_1.wav (deflated 4%)
  adding: augmentation_recs/7_khaled_1_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/8_gian_0_noise_2.wav (deflated 4%)
  adding: augmentation_recs/1_gian_1_noise_0.wav (deflated 4%)
  adding: augmentation_recs/7_gian_3_pitch_1.wav (deflated 7%)
  adding: augmentation_recs/9_khaled_7_pitch_4.wav (deflated 7%)
  adding: augmentation_recs/0_khaled_4_noise_3.wav (deflated 4%)
  adding: augmentation_recs/3_khaled_2_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/9_khaled_3_noise_1.wav (deflated 4%)
  adding: augmentation_recs/5_khaled_8_noise_4.wav (deflated 4%)
  adding: augmentation_recs/7_gian_7_noise_4.wav (deflated 4%)
  adding: augmentation_recs/7_alinda_6_noise_1.wav (deflated 4%)
  adding: augmentation_recs/4_alinda_0_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/9_gia

  adding: augmentation_recs/2_alinda_7_pitch_3.wav (deflated 6%)
  adding: augmentation_recs/1_alinda_1_noise_2.wav (deflated 4%)
  adding: augmentation_recs/8_alinda_6_noise_0.wav (deflated 4%)
  adding: augmentation_recs/4_alinda_9_pitch_0.wav (deflated 6%)
  adding: augmentation_recs/5_khaled_1_noise_4.wav (deflated 4%)
  adding: augmentation_recs/5_khaled_5_pitch_1.wav (deflated 6%)
  adding: augmentation_recs/6_khaled_3_noise_0.wav (deflated 4%)
  adding: augmentation_recs/8_gian_9_noise_2.wav (deflated 4%)
  adding: augmentation_recs/1_gian_8_noise_0.wav (deflated 4%)
  adding: augmentation_recs/1_khaled_2_noise_2.wav (deflated 4%)
  adding: augmentation_recs/2_khaled_4_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/7_khaled_8_pitch_4.wav (deflated 6%)
  adding: augmentation_recs/8_khaled_5_noise_0.wav (deflated 4%)
  adding: augmentation_recs/2_gian_4_noise_2.wav (deflated 4%)
  adding: augmentation_recs/4_gian_6_pitch_3.wav (deflated 6%)
  adding: augmentation_recs/5_ali

  adding: augmentation_recs/7_gian_6_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/5_khaled_9_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/0_gian_6_noise_4.wav (deflated 4%)
  adding: augmentation_recs/0_gian_2_pitch_1.wav (deflated 7%)
  adding: augmentation_recs/9_gian_3_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/7_alinda_3_noise_2.wav (deflated 4%)
  adding: augmentation_recs/4_alinda_5_pitch_3.wav (deflated 7%)
  adding: augmentation_recs/1_alinda_9_pitch_4.wav (deflated 7%)
  adding: augmentation_recs/6_gian_0_noise_0.wav (deflated 4%)
  adding: augmentation_recs/3_alinda_4_pitch_1.wav (deflated 6%)
  adding: augmentation_recs/0_alinda_2_noise_0.wav (deflated 4%)
  adding: augmentation_recs/9_alinda_5_noise_2.wav (deflated 4%)
  adding: augmentation_recs/3_alinda_0_noise_4.wav (deflated 4%)
  adding: augmentation_recs/8_khaled_9_noise_3.wav (deflated 4%)
  adding: augmentation_recs/3_gian_1.wav (deflated 40%)
  adding: augmentation_recs/8_gian_9.wav (de

  adding: augmentation_recs/2_khaled_8_pitch_2.wav (deflated 6%)
  adding: augmentation_recs/3_gian_3.wav (deflated 40%)
  adding: augmentation_recs/4_khaled_2_noise_4.wav (deflated 4%)
  adding: augmentation_recs/8_khaled_9_noise_1.wav (deflated 4%)
  adding: augmentation_recs/2_gian_8_noise_3.wav (deflated 4%)
  adding: augmentation_recs/7_khaled_0_noise_0.wav (deflated 4%)
  adding: augmentation_recs/4_khaled_6_pitch_1.wav (deflated 6%)
  adding: augmentation_recs/6_alinda_8_pitch_4.wav (deflated 5%)
  adding: augmentation_recs/9_alinda_5_noise_0.wav (deflated 4%)
  adding: augmentation_recs/0_alinda_2_noise_2.wav (deflated 4%)
  adding: augmentation_recs/3_alinda_4_pitch_3.wav (deflated 6%)
  adding: augmentation_recs/9_gian_7_noise_4.wav (deflated 4%)
  adding: augmentation_recs/4_alinda_1_noise_4.wav (deflated 4%)
  adding: augmentation_recs/6_gian_0_noise_2.wav (deflated 4%)
  adding: augmentation_recs/4_alinda_5_pitch_1.wav (deflated 7%)
  adding: augmentation_recs/7_alinda_3_n